In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from importlib import reload
sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os
import xgboost as xgb
sys.path.append("./src")

from toolbox import print_progress_bar

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# select model 
model_folder_name = '20230707_140403'
# model_folder_name = '20230706_141045'

# load setting from data/model/model_folder_name/setting.json
setting = ut.load_json(f'./data/model/{model_folder_name}/setting.json')

print(setting.keys())
print(setting['factor_columns'])

dict_keys(['start_index', 'rebalance_date_lst', 'factor_columns', 'target_cols', 'model_save_path', 'param_search'])
['asset_qoq', 'asset_yoy', 'ni_qoq', 'ni_yoy', 'roe', 'roe_yoy', 'roe_4q_sum', 'roe_4q_sum_yoy', 'tobins_q', 'ocf / asset', '20_d_return', '40_d_return', '60_d_return', 'dividend_1Y_sum_yield', 'dividend_2Y_sum_yield', 'dividend_3Y_sum_yield', 'last_dividend_yield']


In [3]:
# create a list for model json file in model/model_folder_name
model_list = []
for model in os.listdir(f'./models/{model_folder_name}'):
    if model.endswith('.json'):
        # append the model path to model_list
        model_list.append(f'./models/{model_folder_name}/{model}')
# sort model_list
model_list.sort()

In [4]:
# load df_factor from data/model/model_folder_name/df_factor_all.feather
df_factor_all = pd.read_feather(f'./data/model/{model_folder_name}/df_factor_all.feather')

In [5]:
# load df_factor from data/model/model_folder_name/df_factor_all.feather
df_factor_all = pd.read_feather(f'./data/model/{model_folder_name}/df_factor_all.feather')

df_model_mapping = pd.DataFrame()
df_model_mapping['rebalance_date'] = setting['rebalance_date_lst'][setting['start_index'] + 0:]
df_model_mapping['model_path'] = model_list
# convert rebalance_date to datetime
df_model_mapping['rebalance_date'] = pd.to_datetime(df_model_mapping['rebalance_date'])

# merge the df_model_mapping and df_factor_all
df_factor_all = pd.merge(df_factor_all, df_model_mapping, left_on='date', right_on='rebalance_date', how='left')
df_factor_all = df_factor_all.sort_values(['date','ticker']).reset_index(drop=True)
df_factor_all['model_path'].fillna(method='ffill', inplace=True)
df_factor_all['rebalance_date'].fillna(method='ffill', inplace=True)
df_factor_all.dropna(subset=['model_path'], inplace=True)

df_factor_all = df_factor_all.reset_index(drop=True)
df_factor_all.to_feather(f'./data/model/{model_folder_name}/df_factor_all_select.feather')

In [6]:
factor_columns = setting['factor_columns']

In [7]:
model_list = df_factor_all['model_path'].unique()
reb_lst = df_factor_all['rebalance_date'].unique()

In [8]:
df_predict_all = pd.DataFrame([])
df_feature_importance_all = pd.DataFrame([])
for i, model_path in enumerate(model_list):
    model = xgb.Booster()
    model.load_model(model_path)
    try:
        df_feature_importance = pd.DataFrame(model.get_score(importance_type='gain').items(), columns=['feature','importance'])
        df_feature_importance.sort_values('importance', ascending=False, inplace=True)
        df_feature_importance['date'] = reb_lst[i]
        df_feature_importance_all = pd.concat([df_feature_importance_all, df_feature_importance], axis=0)

        df_predict = df_factor_all[df_factor_all['rebalance_date'] == reb_lst[i]].reset_index(drop=True).copy()
        df_predict['predict'] = model.predict(xgb.DMatrix(df_predict[factor_columns]))
        df_predict_all = pd.concat([df_predict_all, df_predict], axis=0)
    except:
        print(reb_lst)

In [9]:
df_feature_importance_all.groupby('feature')['importance'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
feature,,,,,,,,
roe_4q_sum,72.0000,90.8868,15.3270,55.3793,84.2963,88.9944,96.4573,157.5303
roe,72.0000,89.4749,33.4784,24.8000,63.6798,84.7111,108.8865,152.7685
dividend_1Y_sum_yield,68.0000,72.8021,34.4652,9.3981,40.4383,75.2050,101.8024,132.5691
40_d_return,72.0000,62.8649,17.0795,16.8763,48.8128,68.0635,75.6197,88.1644
dividend_3Y_sum_yield,71.0000,61.2035,35.7426,16.4021,29.8510,56.9943,81.8929,140.4088
ni_qoq,72.0000,52.5591,41.0599,11.4011,33.1635,37.9390,47.9183,186.6387
tobins_q,72.0000,49.5149,25.8079,19.3535,34.2104,45.4024,62.2609,194.2917
roe_yoy,72.0000,48.9893,48.3559,9.5874,21.3842,35.2456,50.6287,256.6247
60_d_return,72.0000,42.4898,11.8096,20.0835,34.4129,42.0771,47.3561,77.8195


In [10]:
i = 0

df_predict_describe_all = pd.DataFrame([])

for i in range(len(reb_lst)):
    df_predict = df_predict_all[df_predict_all['rebalance_date'] == reb_lst[i]].reset_index(drop=True).copy()
    # calc predict qcut
    df_predict['predict_qcut'] = pd.qcut(df_predict['predict'], 10, labels=False, duplicates='drop')
    df_predict_describe = df_predict.groupby('predict_qcut')['y'].describe().reset_index()
    df_predict_describe['date'] = reb_lst[i]
    df_predict_describe_all = pd.concat([df_predict_describe_all, df_predict_describe], axis=0)

In [11]:
df_predict_describe_all.groupby('predict_qcut')['mean'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
predict_qcut,,,,,,,,
9,64.0000,0.0781,0.1378,-0.2923,0.0136,0.0686,0.1064,0.7397
8,66.0000,0.0418,0.1419,-0.2599,-0.0288,0.0109,0.0723,0.7588
7,68.0000,0.0270,0.1298,-0.2125,-0.0339,0.0002,0.0520,0.6956
6,71.0000,0.0155,0.1187,-0.2748,-0.0455,-0.0059,0.0460,0.5835
5,72.0000,0.0064,0.1167,-0.2852,-0.0491,-0.0170,0.0391,0.5667
4,72.0000,0.0004,0.1130,-0.2847,-0.0658,-0.0227,0.0252,0.5276
3,72.0000,-0.0059,0.1064,-0.2676,-0.0525,-0.0261,0.0134,0.5099
2,72.0000,-0.0128,0.1004,-0.2758,-0.0561,-0.0267,0.0092,0.4607
1,72.0000,-0.0144,0.0999,-0.2657,-0.0561,-0.0345,0.0117,0.4347


In [12]:
df_predict_all = df_predict_all[df_predict_all['date'].isin(reb_lst)]
# df_predict_all['predict_group'] = df_predict_all.groupby('rebalance_date',group_keys=False)['predict'].apply(lambda x :pd.qcut(x, 5, labels=False))
df_predict_all['predict_rank'] = df_predict_all.groupby('rebalance_date',group_keys=False)['predict'].rank(ascending=False)
df_select = df_predict_all[df_predict_all['predict_rank']<=50]
df_select_return = df_select.groupby('date')['y'].mean()


def calculate_cagr(df):
    # Add 1 to all the quarterly returns
    df_plus_one = df + 1
    # Calculate the cumulative product of the returns
    cum_product = df_plus_one.cumprod()
    # Get the total number of years
    num_years = len(df) / 4
    # Calculate CAGR
    cagr = (cum_product.iloc[-1])**(1/num_years) - 1
    return cagr


In [13]:
print(f"SR: {((df_select_return + 1).prod()**(4/len(df_select_return)) -1 ) / (df_select_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_select_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_select_return.std() * np.sqrt(4)):.2%}")

SR: 1.27
CAGR: 40.86%
Std: 32.10%


In [31]:
dividend_min = 0.05
df_dividend_filter = df_predict_all[df_predict_all['last_dividend_yield'] >= dividend_min].copy()
df_predict_filter = df_predict_all[df_predict_all['predict_rank'] <= 300]
df_dividend_select = pd.concat([df_dividend_filter, df_predict_filter]).drop_duplicates().reset_index(drop=True)
df_dividend_select.sort_values(['date','ticker'], inplace=True)

## Consider Liquidity

In [15]:
# df_price_liquidity_flag.to_feather('./data/df_price_liquidity_flag.feather')
df_price_liquidity_flag = pd.read_feather('./data/df_price_liquidity_flag.feather')
df_price_liquidity_flag = df_price_liquidity_flag[['日期','股票代號','flag_all']]
df_price_liquidity_flag.columns = ['date','ticker','flag_all']


df_dividend_select = pd.merge(df_dividend_select, df_price_liquidity_flag, on=['date','ticker'], how='left')
df_dividend_select = df_dividend_select[df_dividend_select['flag_all'] != 1].reset_index(drop=True)


In [16]:
def top_n(df, n=5, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]):
    return df.sort_values(by=columns, ascending=ascending).head(n)


df_dividend_select = df_dividend_select.reset_index(drop=True)
df_dividend_select_top_50 = df_dividend_select.groupby('date').apply(top_n, n=50, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True])

df_dividend_select_top_50.reset_index(drop=True, inplace=True)
df_dividend_select_top_50_return = df_dividend_select_top_50.groupby('date')['y'].mean()
print(f"SR: {((df_dividend_select_top_50_return + 1).prod()**(4/len(df_dividend_select_top_50_return)) -1 ) / (df_dividend_select_top_50_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_dividend_select_top_50_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_dividend_select_top_50_return.std() * np.sqrt(4)):.2%}")


SR: 0.78
CAGR: 23.53%
Std: 30.26%


In [17]:
df_dividend_select.to_feather(f'./data/model/{model_folder_name}/df_dividend_select.feather')

## Consider Turnover

In [46]:
apply_buffer_index = 0
base_portfolio_number = 50
buffer_portfolio_number = 150

In [48]:
df_dividend_select = df_dividend_select.reset_index(drop=True)
df_dividend_select_base = df_dividend_select.groupby('date').apply(top_n, n = base_portfolio_number, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)
df_dividend_select_buffer = df_dividend_select.groupby('date').apply(top_n, n = buffer_portfolio_number, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)

df_dividend_select_buffer.reset_index(drop=True, inplace=True)


apply_buffer_date = np.sort(df_dividend_select_buffer['rebalance_date'].unique())[apply_buffer_index]
# string format
apply_buffer_date = pd.to_datetime(apply_buffer_date).strftime('%Y-%m-%d')
print(f'自 {apply_buffer_date} 開始使用 buffer 建構投組')

df_buffer_result = pd.DataFrame([])
# concat the base before apply_buffer_date to df_buffer_result
df_buffer_result = pd.concat([df_buffer_result, df_dividend_select_base[df_dividend_select_base['date'] < apply_buffer_date]], axis=0)
# print the max date in df_buffer_result
print(f"df_buffer_result['date'].max(): {df_buffer_result['date'].max()}")

for i in range(apply_buffer_index, len(reb_lst)):
    df_dividend_select_buffer_temp = df_dividend_select_buffer[df_dividend_select_buffer['rebalance_date'] == reb_lst[i]].reset_index(drop=True)
    df_buffer_result_temp = df_buffer_result[df_buffer_result['rebalance_date'] == reb_lst[i - 1]].reset_index(drop=True)

    df_new_temp = df_dividend_select_buffer_temp[['date','ticker']].copy()
    df_new_temp['buffer'] = 1
    df_old_temp = df_buffer_result_temp[['date','ticker']].copy()
    df_old_temp['last_portfolio'] = 1

    df_joint_temp = pd.merge(df_old_temp[['ticker','last_portfolio']], df_new_temp[['ticker','buffer']], on=['ticker'], how='outer')
    df_joint_temp['buffer'].fillna(0, inplace=True)
    df_joint_temp['last_portfolio'].fillna(0, inplace=True)
    df_joint_temp['total'] = df_joint_temp['buffer'] + df_joint_temp['last_portfolio']
    df_joint_temp = df_joint_temp[df_joint_temp['total'] == 2]

    joint_ticker = df_joint_temp['ticker'].unique()

    df_priority = df_dividend_select_buffer_temp[df_dividend_select_buffer_temp['ticker'].isin(joint_ticker)]
    df_triviality = df_dividend_select_buffer_temp[df_dividend_select_buffer_temp['ticker'].isin(joint_ticker) == False]
    df_triviality = df_triviality.groupby('date').apply(top_n, n = base_portfolio_number - len(df_priority), columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)


    # concat the df_priority and df_triviality
    df_buffer_result_temp = pd.concat([df_priority, df_triviality], axis=0)

    # concat the df_buffer_result_temp to df_buffer_result
    df_buffer_result = pd.concat([df_buffer_result, df_buffer_result_temp], axis=0)

df_buffer_result.reset_index(drop=True, inplace=True)
df_buffer_result_return = df_buffer_result.groupby('date')['y'].mean()

df_buffer_result.to_feather(f'./data/model/{model_folder_name}/df_buffer_result_{apply_buffer_index}_{base_portfolio_number}_{buffer_portfolio_number}.feather')
print(f"SR: {((df_buffer_result_return + 1).prod()**(4/len(df_buffer_result_return)) -1 ) / (df_buffer_result_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_buffer_result_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_buffer_result_return.std() * np.sqrt(4)):.2%}")

自 2005-03-15 開始使用 buffer 建構投組
df_buffer_result['date'].max(): NaT
SR: 0.91
CAGR: 24.75%
Std: 27.15%
